In [74]:
import evebox as box
from tqdm.notebook import tqdm, trange

In [108]:
import importlib
#importlib.reload(box.esi)
#importlib.reload(box.universe)
importlib.reload(box)
importlib.reload(box.market)

<module 'evebox.market' from 'D:\\Daten\\Repos\\tf_playground\\eveplan\\evebox\\market.py'>

In [97]:
uni = box.Universe.from_esi(cache = 'universe.txt', tqdm = tqdm).range('Jita', min_sec = 0.5)

types = [
    t
    for t, v in hs.market_types.items()
    if v["volume"] <= 1e4
]

uni = uni.sub_market(types)
del types

In [83]:
hs = uni.range('Jita', min_sec = 0.5)

types = [
    t
    for t, v in hs.market_types.items()
    if v["volume"] <= 0.01
]

hs2 = hs.sub_market(types)

In [84]:
import pickle

with open('test.bin', 'wb') as f:
    pickle.dump(uni, f)

In [85]:
len(hs2.market_types)

5606

In [109]:
orders = box.load_orders(hs2, tqdm = tqdm)